In [ ]:
from legend_plot_style import LEGENDPlotStyle as lps
lps.use('legend_talks')
import os
import uproot
import json
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
from matplotlib.legend_handler import HandlerLine2D, HandlerTuple

# THINGS YOU CAN CHANGE - 
dataset = "golden" # silver is giving problems, but I'll fix it if needed - not the priority now
det_inspected = "All" # or "" for BEGe/ICPC/... 

In [ ]:
if dataset == "golden":
    if det_inspected != "All":
        RAW_path = f"/lfs/l1/legend/users/calgaro/taup/legend-gamma-lines-analysis/output/2023-08-21/0_25keV/raw"
        AC_path  = f"/lfs/l1/legend/users/calgaro/taup/legend-gamma-lines-analysis/output/2023-08-21/0_25keV/LArAC"
        C_path   = f"/lfs/l1/legend/users/calgaro/taup/legend-gamma-lines-analysis/output/2023-08-21/0_25keV/LArC"
    if det_inspected == "All":
        RAW_path = f"/lfs/l1/legend/users/calgaro/taup/legend-gamma-lines-analysis/output/2023-08-21/0_25keV/raw"
        AC_path  = f"/lfs/l1/legend/users/calgaro/taup/legend-gamma-lines-analysis/output/2023-08-21/0_25keV/LArAC"
        C_path   = f"/lfs/l1/legend/users/calgaro/taup/legend-gamma-lines-analysis/output/2023-08-21/0_25keV/LArC"
else:
    RAW_path = f"/lfs/l1/legend/users/calgaro/taup/legend-gamma-lines-analysis/output/2023-08-17-test/0_25keV/raw"
    AC_path  = f"/lfs/l1/legend/users/calgaro/taup/legend-gamma-lines-analysis/output/2023-08-17-test/0_25keV/LArAC"
    C_path   = f"/lfs/l1/legend/users/calgaro/taup/legend-gamma-lines-analysis/output/2023-08-17-test/0_25keV/LArC"

single_path = f"/lfs/l1/legend/users/calgaro/taup/legend-gamma-lines-analysis/output/2023-08-21/0_25keV/" 
periods = "p3_p4"
if dataset == "golden":
    exposure_path = f"/lfs/l1/legend/users/calgaro/taup/legend-gamma-lines-analysis/livetime_and_exposure/exposure_in_kg_yr_on_{periods}.json"
else:
    exposure_path = f"/lfs/l1/legend/users/calgaro/taup/legend-gamma-lines-analysis/livetime_and_exposure/exposure_in_kg_yr_on_no_psd_{periods}.json"

In [ ]:
from legendmeta import JsonDB
map_file = os.path.join("/lfs/l1/legend/data/public/prodenv/prod-blind/ref/", "v01.06", "inputs/hardware/configuration/channelmaps")
full_det_map = JsonDB(map_file).on(timestamp="20230601T000000Z")
full_det_map = {k:v for k,v in full_det_map.items() if "location" in v.keys() and "S" not in k} # remove SiPMs and AUX/DUMMY channels

# sort the dictionary keys based on location and position
det_list = sorted(full_det_map.keys(), key=lambda key: (full_det_map[key]['location']['string'], full_det_map[key]['location']['position']))
exposure_json = json.load(open(exposure_path))

exp_dict = {}

# integrate exposure for each single detector over runs and periods
for det in det_list:
    tmp_exp = 0
    for period in exposure_json.keys():
        for run in exposure_json[period].keys():
            if det not in exposure_json[period][run].keys():
                tmp_exp += 0 # might be off or ac
            else:
                tmp_exp += exposure_json[period][run][det]
    exp_dict[det] = tmp_exp
    
exp_dict = {k:v for k,v in exp_dict.items() if v != 0} # remove dets for which the exposure is null
print(exp_dict)
len(exp_dict.keys())

# Detector type by detector type

In [ ]:
"""
# p3 (silver)
ICPC_exposure = 5.86061 # kg yr
BEGe_exposure = 1.31589 # kg yr
COAX_exposure = 1.16841 # kg yr
PPC_exposure  = 0.86551 # kg yr
# p4  (silver)
ICPC_exposure = 3.48628 # kg yr
BEGe_exposure = 0.78278 # kg yr
COAX_exposure = 0.69505 # kg yr
PPC_exposure  = 0.55678 # kg yr
"""

# p3 + p4
if dataset == "golden":
    ICPC_exposure = 8.0028 # kg yr
    BEGe_exposure = 2.0987 # kg yr
    COAX_exposure = 0 # kg yr
    PPC_exposure  = 0 # kg yr
else:
    ICPC_exposure = 9.3469 # kg yr
    BEGe_exposure = 2.0987 # kg yr
    COAX_exposure = 1.8635 # kg yr
    PPC_exposure  = 1.4223 # kg yr

dict_exposure = {"ICPC": ICPC_exposure, "BEGe": BEGe_exposure, "COAX": COAX_exposure, "PPC": PPC_exposure}
dict_exposure.update({"All": sum(dict_exposure.values())})
print(dict_exposure)

In [ ]:
gamma     = []
cut       = []
det_type  = []
intensity = []
range_min = []
range_max = []

avoid = ["e+e-_Kr85_514", "Pb212_239_Pb214_242", "Ac228_338_Pb214_352", "Ac228_965_Ac228_969", "Bi214_1120_Zn65_1125"]
for cut_path in [RAW_path, AC_path, C_path]
    if det_inspected != "All":
        for detector_type in ["ICPC", "BEGe", "PPC", "COAX"]:
            if dict_exposure[detector_type] == 0:
                continue
            legend = json.load(open("histo.gamma.json"))
            if legend["name_fit"] not in avoid:
                gamma.append("name_fit")
                det_type.append(detector_type)
                if "raw"  in cut_path: cut.append("raw")
                if "LArAC" in cut_path: cut.append("LArAC")
                if "LArC" in cut_path: cut.append("LArC")
                intensity_keys = [k for keys in legend["fit_parameters"]["line"].keys() if "intensity" in keys]:
                for int_key in intensity_keys: 
                    try:
                        intensity.append(legend["fit_parameters"]["line"][int_key]["mode"]/dict_exposure[detector_type])
                        range_min.append(legend["fit_parameters"]["line"][int_key]["low"]/dict_exposure[detector_type])
                        range_max.append(legend["fit_parameters"]["line"][int_key]["high"]/dict_exposure[detector_type])
                    except:
                        intensity.append(legend["fit_parameters"]["line"][int_key]["upper_limit"]/dict_exposure[detector_type])
                        range_min.append(0.0)
                        range_max.append(0.0)
    if det_inspected == "All":
        detector_type = "All"
        if dict_exposure[detector_type] == 0:
            continue
        legend = json.load(open("histo.gamma.json"))
            if legend["name_fit"] not in avoid:
                gamma.append("name_fit")
                det_type.append(detector_type)
                if "raw"  in cut_path: cut.append("raw")
                if "LArAC" in cut_path: cut.append("LArAC")
                if "LArC" in cut_path: cut.append("LArC")
                intensity_keys = [k for keys in legend["fit_parameters"]["line"].keys() if "intensity" in keys]:
                for int_key in intensity_keys: 
                    try:
                        intensity.append(legend["fit_parameters"]["line"][int_key]["mode"]/dict_exposure[detector_type])
                        range_min.append(legend["fit_parameters"]["line"][int_key]["low"]/dict_exposure[detector_type])
                        range_max.append(legend["fit_parameters"]["line"][int_key]["high"]/dict_exposure[detector_type])
                    except:
                        intensity.append(legend["fit_parameters"]["line"][int_key]["upper_limit"]/dict_exposure[detector_type])
                        range_min.append(0.0)
                        range_max.append(0.0)   

df = pd.DataFrame()
print(len(gamma))
print(len(det_type))
print(len(cut))
print(len(intensity))
print(len(range_min))
print(len(range_max))
df["gamma"]     = gamma
df["det_type"]  = det_type
df["cut"]       = cut
df["intensity"] = intensity
df["range_min"] = range_min
df["range_max"] = range_max

df_2 = df.copy()
df_2["yerr_low"] = df_2.apply(lambda row: max(row["intensity"] - row["range_min"], 0), axis=1)
df_2["yerr_upp"] = df_2.apply(lambda row: max(-row["intensity"] + row["range_max"], 0), axis=1)
df_2.loc[df_2["intensity"] == df_2["yerr_low"], "yerr_low"] = 0
if det_inspected != "All": det_type_order = ["BEGe", "ICPC", "PPC", "COAX"]
if det_inspected == "All": det_type_order = ["All"]
df_2['det_type'] = pd.Categorical(df_2['det_type'], categories=det_type_order, ordered=True)
df_2 = df_2.sort_values("det_type")
df = df_2.copy()
df

In [ ]:
#for retrieving some output
df_2[(df_2["gamma"]=="Bi214_2204") & (df_2["cut"]=="raw")]

color_cut = {"raw" : "#07A9FF", "LArAC": "red", "LArC" : "orange"}
legend_dict = {"raw" : "raw", "LArAC": "LAr AC", "LArC" : "LAr C"}
cut_order = ["raw", "LArC", "LArAC"]

# Group the data by gamma and cut
grouped = df.groupby(['gamma', 'cut'])

if det_inspected != "All":
    for g in df.gamma.unique():
        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 5))  # Create a 1x3 grid of subplots
        for idx, cut in enumerate(cut_order):
            ax = axes[idx]  # Get the current subplot
            ax.set_title(f'Cut: {cut}')  # Set the title

            # Plot error bars
            for det_type in df.det_type.unique():
                condition = (df.gamma == g) & (df.cut == cut) & (df.det_type == det_type)
                yerr_low_zero = (df[condition]["yerr_low"] == 0).all()
                yerr_upp_zero = (df[condition]["yerr_upp"] == 0).all()

                if yerr_low_zero and yerr_upp_zero:
                    ax.plot(df[condition].det_type, 
                             df[condition].intensity, 
                             marker="x", color=color_cut[cut], markersize=8)
                else:
                    ax.errorbar(
                        df[condition].det_type,
                        df[condition].intensity,
                        yerr=[df[condition].yerr_low, df[condition].yerr_upp],
                        marker=".", linestyle="", color=color_cut[cut], markersize=8
                    )

            ax.set_xlabel('Detector Type')
            ax.set_ylabel('gamma line rate [counts/(kg yr)]')

        fig.suptitle(f"Line: {g}")
        plt.show()
        #plt.savefig(f"{dataset}/{g}.pdf")
        #plt.savefig(f"{dataset}/{g}.png")
else:
    gamma_title_dict = {
         'K42_1525': "1525 (K-42)",
         'K40_1461': "1461 (K-40)",
         'Co60_1332': "1332 (Co-60)",
         'Co60_1173': "1173 (Co-60)",
         'Ac228_911': "911 (Ac-228)",
         'Bi212_727': "727 (Bi-212)",
         'Tl208_2614': "2614 (Tl-208)",
         'Tl208_583': "583 (Tl-208)",
         'Tl208_861': "861 (Tl-208)",
         'Pa234m_1001': "1001 (Pa-234m)",
         'Pb214_352': "352 (Pb-214)",
         'Pb214_295': "295 (Pb-214)",
         'Bi214_609': "609 (Bi-214)",
         'Bi214_1378': "1378 (Bi-214)",
         'Bi214_1764': "1764 (Bi-214)",
         'Bi214_1238': "1238 (Bi-214)",
         'Bi214_2204': "2204 (Bi-214)",
         'Bi214_2448': "2448 (Bi-214)"
    }
    sorted_gamma = sorted(df.gamma.unique(), key=lambda x: int(x.split("_")[1]))
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8,9))  
    axes[0].set_position([0.1, 0.1, 0.6, 0.85])  # [left, bottom, width, height]
    axes[1].set_position([0.76, 0.1, 0.2, 0.85])
    ax = axes[0]
    for g in sorted_gamma:
        if g in ["K40_1461", "K42_1525"]:
            continue
        for idx, cut in enumerate(cut_order):
            ax.set_title(f'Cut: {cut}')  # Set the title
            condition = (df.gamma == g) & (df.cut == cut) & (df.det_type == det_type)
            yerr_low_zero = (df[condition]["yerr_low"] == 0).all()
            yerr_upp_zero = (df[condition]["yerr_upp"] == 0).all()

            if yerr_low_zero and yerr_upp_zero:
                ax.plot(gamma_title_dict[g], 
                         df[condition].intensity, 
                         marker="x", color=color_cut[cut], markersize=8)
            else:
                ax.errorbar(
                    gamma_title_dict[g],
                    df[condition].intensity,
                    yerr=[df[condition].yerr_low, df[condition].yerr_upp],
                    marker=".", linestyle="", color=color_cut[cut], markersize=8
                )
    ax.set_ylabel('gamma line rate [counts/(kg yr)]')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='center')
                
    ax = axes[1]
    #ax.set_legend_logo(position='upper right')
    for g in sorted_gamma:
        if g not in ["K40_1461", "K42_1525"]:
            continue
        for idx, cut in enumerate(cut_order):
            ax.set_title(f'Cut: {cut}')  # Set the title

            condition = (df.gamma == g) & (df.cut == cut) & (df.det_type == det_type)
            yerr_low_zero = (df[condition]["yerr_low"] == 0).all()
            yerr_upp_zero = (df[condition]["yerr_upp"] == 0).all()

            if yerr_low_zero and yerr_upp_zero:
                ax.plot(gamma_title_dict[g], 
                         df[condition].intensity, 
                         marker="x", color=color_cut[cut], markersize=8)
            else:
                ax.errorbar(
                    gamma_title_dict[g],
                    df[condition].intensity,
                    yerr=[df[condition].yerr_low, df[condition].yerr_upp],
                    marker=".", linestyle="", color=color_cut[cut], markersize=8
                )

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='center')
    
    axes[1].set_legend_annotation()
    marker_proxy_dot = mlines.Line2D([], [], linestyle='-', marker='.', color='black', markersize=10, label='global mode +- 68%')
    marker_proxy_v = mlines.Line2D([], [], linestyle='None', marker='v', color='black', markersize=10, label='upper 90% CI limit')
    marker_proxy_dot_raw = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["raw"], markersize=10, label='raw')
    marker_proxy_v_raw = mlines.Line2D([], [], linestyle='None', marker='v', color=color_cut["raw"], markersize=10, label='raw')
    marker_proxy_dot_LArC = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["LArC"], markersize=10, label='LAr C')
    marker_proxy_v_LArC = mlines.Line2D([], [], linestyle='None', marker='v', color=color_cut["LArC"], markersize=10, label='LAr C')
    marker_proxy_dot_LArAC = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["LArAC"], markersize=10, label='LAr AC')
    marker_proxy_v_LArAC = mlines.Line2D([], [], linestyle='None', marker='v', color=color_cut["LArAC"], markersize=10, label='LAr AC')
    axes[0].legend([(marker_proxy_dot_raw, marker_proxy_v_raw), (marker_proxy_dot_LArAC, marker_proxy_v_LArAC), (marker_proxy_dot_LArC, marker_proxy_v_LArC), marker_proxy_dot, marker_proxy_v], 
            ["raw", 'LAr AC', "LAr C", 'global mode +- 68%', 'upper 90% limit'], 
            numpoints=1, 
            handler_map={tuple: HandlerTuple(ndivide=None)},
            loc = "upper right")

    fig.suptitle(f"{dataset} dataset")
    plt.show()
    #plt.savefig(f"{dataset}/All.pdf")
    #plt.savefig(f"{dataset}/All.png")

### ...GERDA?

In [ ]:
with open('../gerda_results/phII/BEGe_II.json', 'r') as json_file:
    GERDA_BEGe_phII = json.load(json_file)
with open('../gerda_results/phIIplus/BEGe_IIplus.json', 'r') as json_file:
    GERDA_BEGe_phII_plus = json.load(json_file)
with open('../gerda_results/phII/COAX_phII.json', 'r') as json_file:
    GERDA_COAX_phII = json.load(json_file)
with open('../gerda_results/phIIplus/COAX_phIIplus.json', 'r') as json_file:
    GERDA_COAX_phII_plus = json.load(json_file)
with open('../gerda_results/phIIplus/ICPC_phIIplus.json', 'r') as json_file:
    GERDA_IC_phII_plus = json.load(json_file)

color_cut = {"raw" : "#07A9FF", "LArAC": "red", "LArC" : "orange"}
legend_dict = {"raw" : "raw", "LArAC": "LAr AC", "LArC" : "LAr C"}
cut_order = ["raw", "LArC", "LArAC"]

# Group the data by gamma and cut
grouped = df.groupby(['gamma', 'cut'])

import re
for g in df.gamma.unique():
    isotope_num = g.split("_")[0]
    num = re.findall(r'\d+\.\d+|\d+', isotope_num)[0]
    isotope = isotope_num.split(num)[0]
    first_energy = g.split("_")[1]
    element_to_find = int(first_energy)  
    
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 5))  # Create a 1x3 grid of subplots
    for idx, cut in enumerate(cut_order):
        ax = axes[idx]  # Get the current subplot
        ax.set_title(f'Cut: {cut}')  # Set the title
        
        if det_inspected != "All":
            # Plot error bars
            for det_type in ["BEGe", "ICPC", "PPC", "COAX"]:
                if dict_exposure[detector_type] == 0:
                    continue
                condition = (df.gamma == g) & (df.cut == cut) & (df.det_type == det_type)
                yerr_low_zero = (df[condition]["yerr_low"] == 0).all()
                yerr_upp_zero = (df[condition]["yerr_upp"] == 0).all()

                if yerr_low_zero and yerr_upp_zero:
                    ax.plot(df[condition].det_type, 
                             df[condition].intensity, 
                             marker="x", color=color_cut[cut], markersize=8)
                else:
                    ax.errorbar(
                        df[condition].det_type,
                        df[condition].intensity,
                        yerr=[df[condition].yerr_low, df[condition].yerr_upp],
                        marker=".", linestyle="", color=color_cut[cut], markersize=8
                    )
        else:
            detector_type = det_type = "All"
            if dict_exposure[detector_type] == 0:
                continue
            condition = (df.gamma == g) & (df.cut == cut) & (df.det_type == det_type)
            yerr_low_zero = (df[condition]["yerr_low"] == 0).all()
            yerr_upp_zero = (df[condition]["yerr_upp"] == 0).all()

            if yerr_low_zero and yerr_upp_zero:
                ax.plot(df[condition].det_type, 
                         df[condition].intensity, 
                         marker="x", color=color_cut[cut], markersize=8)
            else:
                ax.errorbar(
                    df[condition].det_type,
                    df[condition].intensity,
                    yerr=[df[condition].yerr_low, df[condition].yerr_upp],
                    marker=".", linestyle="", color=color_cut[cut], markersize=8)
                
        # Plot error bars
        for det_type in ["BEGe", "COAX", "ICPC", "PPC"]:
            if dict_exposure[detector_type] == 0:
                continue
            condition = (df.gamma == g) & (df.cut == cut) & (df.det_type == det_type)    
            if det_type != "PPC":
                gerda_II = {}
                gerda_II_plus = {}
                if det_type == "BEGe":
                    gerda_II = GERDA_BEGe_phII
                    gerda_II_plus = GERDA_BEGe_phII_plus
                if det_type == "COAX":
                    gerda_II = GERDA_COAX_phII
                    gerda_II_plus = GERDA_COAX_phII_plus
                if det_type == "ICPC":
                    gerda_II_plus = GERDA_IC_phII_plus
                    
                if det_type != "ICPC":
                    if element_to_find in list(gerda_II["energy"]):
                        index_II = gerda_II["energy"].index(element_to_find)
                        if gerda_II[cut]["low_68"][index_II] == 0 and gerda_II[cut]["upp_68"][index_II] == 0:
                            ax.plot(f"{det_type} II", gerda_II[cut]["mode"][index_II], marker="x", color=color_cut[cut], markersize=8)
                        else:
                            ax.errorbar(
                                f"{det_type} II",
                                gerda_II[cut]["mode"][index_II],
                                yerr=[[gerda_II[cut]["low_68"][index_II]], [gerda_II[cut]["upp_68"][index_II]]],
                                marker="s", linestyle="", color=color_cut[cut], markersize=5
                            )
                if element_to_find in list(gerda_II_plus["energy"]):
                    index_II_plus = gerda_II_plus["energy"].index(element_to_find)
                    if gerda_II_plus[cut]["low_68"][index_II_plus] == 0 and gerda_II_plus[cut]["upp_68"][index_II_plus] == 0:
                        ax.plot(f"{det_type} II+", gerda_II_plus[cut]["mode"][index_II_plus], marker="x", color=color_cut[cut], markersize=8)
                    else:
                        ax.errorbar(
                            f"{det_type} II+",
                            gerda_II_plus[cut]["mode"][index_II_plus],
                            yerr=[[gerda_II_plus[cut]["low_68"][index_II_plus]], [gerda_II_plus[cut]["upp_68"][index_II_plus]]],
                            marker="s", linestyle="", color=color_cut[cut], markersize=5
                        )
                    if det_inspected != "All":
                        ax.axvline(1.5, linestyle='--', color='gray')
                    else:
                        ax.axvline(0.5, linestyle='--', color='gray')
                    ax.text(
                        0.81, 1.07, f"GERDA",  # Text position (normalized coordinates)
                        transform=ax.transAxes,  # Coordinate system of the text position
                        fontsize=10, color=color_cut[cut],
                        va="top", ha="left",
                        bbox=dict(boxstyle="round,pad=0.3", edgecolor="gray", facecolor="white")
                    )

        ax.set_xlabel('Detector Type')
        ax.set_ylabel('gamma line rate [counts/(kg yr)]')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='center')
        
        ax.text(
            0.05, 1.07, f"LEGEND",  # Text position (normalized coordinates)
            transform=ax.transAxes,  # Coordinate system of the text position
            fontsize=10, color=color_cut[cut],
            va="top", ha="left",
            bbox=dict(boxstyle="round,pad=0.3", edgecolor="gray", facecolor="white")
        )

    fig.suptitle(f"Line: {g}")
    plt.show()
    #plt.savefig(f"{dataset}/comparison_with_GERDA/{g}.pdf")
    #plt.savefig(f"{dataset}/comparison_with_GERDA/{g}.png")


# Double lines

In [ ]:
gamma3     = []
cut3       = []
det_type3  = []
intensity3 = []
range_min3 = []
range_max3 = []

for cut_path in [RAW_path, AC_path, C_path]:
    
    if det_inspected != "All":
        for detector_type in ["ICPC", "BEGe", "COAX", "PPC"]:
            if dict_exposure[detector_type] == 0:
                continue
            with open(os.path.join(cut_path, detector_type, "histo.gamma.log")) as f:
                f = f.readlines()
                
            for index in range(len(f)):
                if ("*" in f[index]):
                    if f[index].split(" ")[1] in ["e+e-_Kr85_514", "Pb212_239_Pb214_242", "Ac228_338_Pb214_352", "Ac228_965_Ac228_969", "Bi214_1120_Zn65_1125"]:
                        gamma3.append(f[index].split(" ")[1])
                        det_type3.append(detector_type)
                        if "raw"  in cut_path: cut3.append("raw")
                        if "LArAC" in cut_path: cut3.append("LArAC")
                        if "LArC" in cut_path: cut3.append("LArC")
                        for sub_index in range(17):
                            if "intensity" in f[index + sub_index]:
                                try:
                                    intensity3.append(float(f[index + sub_index].split(" ")[9][1:])/dict_exposure[detector_type])
                                    range_min3.append(float(f[index + sub_index].split(" ")[10].split(",")[0][1:])/dict_exposure[detector_type])
                                    range_max3.append(float(f[index + sub_index].split(" ")[10].split(",")[1][:-2])/dict_exposure[detector_type])
                                except: 
                                    intensity3.append(float(f[index+ sub_index].split(" ")[10][1:])/dict_exposure[detector_type])
                                    range_min3.append(0.0)
                                    range_max3.append(0.0)                

                    else:
                        continue
                        
    if det_inspected == "All":
        detector_type = "All"
        if dict_exposure[detector_type] == 0:
            continue
        with open(os.path.join(cut_path, detector_type, "histo.gamma.log")) as f:
            f = f.readlines()

        for index in range(len(f)):
            if ("*" in f[index]):
                if f[index].split(" ")[1] in ["e+e-_Kr85_514", "Pb212_239_Pb214_242", "Ac228_338_Pb214_352", "Ac228_965_Ac228_969", "Bi214_1120_Zn65_1125"]:
                    gamma3.append(f[index].split(" ")[1])
                    det_type3.append(detector_type)
                    if "raw"  in cut_path: cut3.append("raw")
                    if "LArAC" in cut_path: cut3.append("LArAC")
                    if "LArC" in cut_path: cut3.append("LArC")
                    for sub_index in range(17):
                        if "intensity" in f[index + sub_index]:
                            try:
                                intensity3.append(float(f[index + sub_index].split(" ")[9][1:])/dict_exposure[detector_type])
                                range_min3.append(float(f[index + sub_index].split(" ")[10].split(",")[0][1:])/dict_exposure[detector_type])
                                range_max3.append(float(f[index + sub_index].split(" ")[10].split(",")[1][:-2])/dict_exposure[detector_type])
                            except: 
                                intensity3.append(float(f[index+ sub_index].split(" ")[10][1:])/dict_exposure[detector_type])
                                range_min3.append(0.0)
                                range_max3.append(0.0)                

                else:
                    continue
df3 = pd.DataFrame()

gamma3 = [item for item in gamma3 for _ in range(2)]
det_type3 = [item for item in det_type3 for _ in range(2)]
cut3 = [item for item in cut3 for _ in range(2)]

print(len(gamma3))
print(len(det_type3))
print(len(cut3))
print(len(intensity3))
print(len(range_min3))
print(len(range_max3))

df3["gamma"]     = gamma3
df3["det_type"]  = det_type3
df3["cut"]       = cut3
df3["intensity"] = intensity3
df3["range_min"] = range_min3
df3["range_max"] = range_max3

df3["yerr_low"] = df3.apply(lambda row: max(row["intensity"] - row["range_min"], 0), axis=1)
df3["yerr_upp"] = df3.apply(lambda row: max(-row["intensity"] + row["range_max"], 0), axis=1)
df3.loc[df3["intensity"] == df3["yerr_low"], "yerr_low"] = 0

first_peak = df3[df3.index % 2 == 0]
second_peak = df3[df3.index % 2 != 0]

first_peak = first_peak.reset_index(drop=True)
det_type_order = ["BEGe", "ICPC"] if det_inspected != "All" else ["All"]
first_peak['det_type'] = pd.Categorical(first_peak['det_type'], categories=det_type_order, ordered=True)
first_peak = first_peak.sort_values("det_type")

second_peak = second_peak.reset_index(drop=True)
det_type_order = ["BEGe", "ICPC"] if det_inspected != "All" else ["All"]
second_peak['det_type'] = pd.Categorical(second_peak['det_type'], categories=det_type_order, ordered=True)
second_peak = second_peak.sort_values("det_type")

In [ ]:
#flag = "e+e-_Kr85_514"
flag = "Pb212_239_Pb214_242"
#flag = "Ac228_338_Pb214_352"
#flag = "Ac228_965_Ac228_969"
#flag = "Bi214_1120_Zn65_1125"
my_cut = "raw"
first_peak[(first_peak["gamma"]==flag) & (first_peak["cut"]==my_cut)]
#second_peak[(second_peak["gamma"]==flag) & (second_peak["cut"]==my_cut)]

In [ ]:
color_cut = {"raw" : "#07A9FF", "LArAC": "red", "LArC" : "orange"}
legend_dict = {"raw" : "raw", "LArAC": "LAr AC", "LArC" : "LAr C"}
cut_order = ["raw", "LArC", "LArAC"]

for peak in ["first", "second"]:
    if peak == "first":
        df3 = first_peak.copy()
    else:
        df3 = second_peak.copy()

    # Group the data by gamma and cut
    grouped = df3.groupby(['gamma', 'cut'])

    for g in df3.gamma.unique():
        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 5))  # Create a 1x3 grid of subplots
        for idx, cut in enumerate(cut_order):
            ax = axes[idx]  # Get the current subplot
            ax.set_title(f'Cut: {cut}')  # Set the title

            # Plot error bars
            if det_inspected != "All":
                dets = df3.det_type.unique()
            else:
                dets = ["All"]
            for det_type in dets:
                condition = (df3.gamma == g) & (df3.cut == cut) & (df3.det_type == det_type)
                yerr_low_zero = (df3[condition]["yerr_low"] == 0).all()
                yerr_upp_zero = (df3[condition]["yerr_upp"] == 0).all()
                x_data = df3[condition].det_type if det_inspected != "All" else [det_type]

                if yerr_low_zero and yerr_upp_zero:
                    ax.plot(x_data, 
                             df3[condition].intensity, 
                             marker="x", color=color_cut[cut], markersize=8)
                else:
                    ax.errorbar(
                        x_data,
                        df3[condition].intensity,
                        yerr=[df3[condition].yerr_low, df3[condition].yerr_upp],
                        marker=".", linestyle="", color=color_cut[cut], markersize=8
                    )


            ax.set_xlabel('Detector Type')
            ax.set_ylabel('gamma line rate [counts/(kg yr)]')
        if "e+" in g:
            if peak == "first": title = "e+e- 511"
            else: title = "Kr85_514"
        if "Pb212_239_Pb214_242" in g:
            if peak == "first": title = "Pb212_239"
            else: title = "Pb214_242"
        if "Ac228_338_Pb214_352" in g:
            if peak == "first": title = "Ac228_338"
            else: title = "Pb214_352"
        if "Ac228_965_Ac228_969" in g:
            if peak == "first": title = "Ac228_965"
            else: title = "Ac228_969"
        if "Bi214_1120_Zn65_1125" in g:
            if peak == "first": title = "Bi214_1120"
            else: title = "Zn65_1125"
        fig.suptitle(f"Line: {title}")
        plt.show()
        #if det_inspected != "All":
        #    plt.savefig(f"{dataset}/{g}_first_peak.pdf")
        #    plt.savefig(f"{dataset}/{g}_first_peak.png")
        #    plt.savefig(f"{dataset}/{g}_second_peak.pdf")
        #    plt.savefig(f"{dataset}/{g}_second_peak.png")
        #else:
        #    plt.savefig(f"{dataset}/All_first_peak.pdf")
        #    plt.savefig(f"{dataset}/All_first_peak.png")
        #    plt.savefig(f"{dataset}/All_second_peak.pdf")
        #    plt.savefig(f"{dataset}/All_second_peak.png")
        

# K lines - det by det

In [ ]:
detector4  = []
intensity4 = []
range_min4 = []
range_max4 = []
gamma4     = []
cut4       = []

for cut_path in os.listdir(single_path):
    cut_path = os.path.join(single_path, cut_path, "potassium_lines")
    for det in os.listdir(os.path.join(cut_path, "single")):

        with open(os.path.join(cut_path, "single", det, "histo.gamma.log")) as f:
            f = f.readlines()
        for index in range(len(f)):
            if "****" in f[index]:
                if "raw"  in cut_path: cut4.append("raw")
                if "LArAC" in cut_path: cut4.append("LArAC")
                if "LArC" in cut_path: cut4.append("LArC")
                gamma4.append(f[index].split(" ")[1])
                detector4.append(det)
                for sub_index in range(10):
                    if "intensity" in f[index + sub_index]:
                        try:
                            intensity4.append(float(f[index + sub_index].split(" ")[9][1:])/exp_dict[det])
                            range_min4.append(float(f[index + sub_index].split(" ")[10].split(",")[0][1:])/exp_dict[det])
                            range_max4.append(float(f[index + sub_index].split(" ")[10].split(",")[1][:-2])/exp_dict[det])
                        except: 
                            intensity4.append(float(f[index+ sub_index].split(" ")[10][1:])/exp_dict[det])
                            range_min4.append(0.0)
                            range_max4.append(0.0)  
                                
print(len(gamma4))
print(len(cut4))
print(len(detector4))
print(len(intensity4))
print(len(range_min4))
print(len(range_max4))

single_df = pd.DataFrame()
single_df["gamma"] = gamma4
single_df["cut"] = cut4
single_df["detector"] = detector4
single_df["intensity"] = intensity4
single_df["range_min"] = range_min4
single_df["range_max"] = range_max4

from legendmeta import JsonDB
map_file = os.path.join("/lfs/l1/legend/data/public/prodenv/prod-blind/ref/v01.06", "inputs/dataprod/config")
full_status_map = JsonDB(map_file).on(
    timestamp="20230701T000000Z", system="geds"
)["analysis"]
status_map = {key: value for key, value in full_status_map.items() if value.get('usability') in ["off", "ac", "no_psd"]}
all_detectors = [det for det in status_map.keys() if "location" in status_map[det].keys() and "S" not in det]# remove SiPMs and AUX/DUMMY channels
map_file = os.path.join("/lfs/l1/legend/data/public/prodenv/prod-blind/ref/v01.06", "inputs/hardware/configuration/channelmaps")
channel_map = JsonDB(map_file).on(timestamp="20230701T000000Z")

def get_position(detector_key):
    return channel_map[detector_key]['location']['position']
def get_string(detector_key):
    return channel_map[detector_key]['location']['string']
def get_expo(detector_key):
    return exp_dict[detector_key]

single_df['string'] = single_df['detector'].apply(get_string)
single_df['position'] = single_df['detector'].apply(get_position)
single_df['expo'] = single_df['detector'].apply(get_expo)
single_df['name'] = "s"+single_df['string'].astype(str)+"-p"+single_df['position'].astype(str)+"-"+single_df['detector']
single_df

In [ ]:
color_cut = {"raw" : lps.colors["legend_blue"], "LArAC":  lps.colors["legend_darkblue"], "LArC" :  lps.colors["legend_orange"]}
legend_dict = {"raw" : "raw", "LArAC": "LAr AC", "LArC" : "LAr C"}
cut_order = ["raw", "LArC", "LArAC"]

peaks = ["K40_1461", "K42_1525"]

for peak in peaks:
    fix, axes = plt.subplots(figsize=(18,8))
    axes.set_legend_logo(position='upper right')

    for cut in ["raw", "LArC", "LArAC"]:
        df = single_df[(single_df.gamma == peak) & (single_df.cut == cut)].copy()

        df["yerr_low"] = df.apply(lambda row: max(row["intensity"] - row["range_min"], 0), axis=1)
        df["yerr_upp"] = df.apply(lambda row: max(-row["intensity"] + row["range_max"], 0), axis=1)
        df.loc[df["intensity"] == df["yerr_low"], "yerr_low"] = 0

        # order detectors based on their position/location in the array
        valid_detectors = [detector for detector in det_list if detector in df["detector"].values]
        # Convert the "detector" column to a categorical data type with the specified order
        df["detector"] = pd.Categorical(df["detector"], categories=valid_detectors, ordered=True)
        # Reorder the DataFrame based on the categorical order
        df = df.sort_values("detector")

        for det in df.detector.values:
            if det == df.detector.values[0]:
                if df[df.detector==det]["yerr_low"].values[0]!=0 and df[df.detector==det]["yerr_upp"].values[0]!=0 and df[df.detector==det]["intensity"].values[0]!=0:
                    plt.errorbar(
                        df[df.detector==det]["name"],
                        df[df.detector==det]["intensity"],
                        yerr=[df[df.detector==det]["yerr_low"], df[df.detector==det]["yerr_upp"]],
                        marker=".", linestyle="", color=color_cut[cut], markersize=5, label=legend_dict[cut], capsize=3
                    )
                else:
                    plt.errorbar(
                        df[df.detector==det]["name"],
                        df[df.detector==det]["intensity"],
                        yerr=[df[df.detector==det]["yerr_low"], df[df.detector==det]["yerr_upp"]],
                        marker="_", uplims = True, linestyle="", color=color_cut[cut], markersize=5, label=legend_dict[cut], capsize=3
                    )
            else:
                if df[df.detector==det]["yerr_low"].values[0]!=0 and df[df.detector==det]["yerr_upp"].values[0]!=0 and df[df.detector==det]["intensity"].values[0]!=0:
                    plt.errorbar(
                        df[df.detector==det]["name"],
                        df[df.detector==det]["intensity"],
                        yerr=[df[df.detector==det]["yerr_low"], df[df.detector==det]["yerr_upp"]],
                        marker=".", linestyle="", color=color_cut[cut], markersize=5, capsize=3
                    )
                else:
                    plt.errorbar(
                        df[df.detector==det]["name"],
                        df[df.detector==det]["intensity"],
                        yerr=[df[df.detector==det]["yerr_low"], df[df.detector==det]["yerr_upp"]],
                        marker="_", uplims = True, linestyle="", color=color_cut[cut], markersize=5, capsize=3
                    )

    # delimit strings
    first_dets_of_a_string = [0, 8, 10, 12, 19, 24, 30, 37, 40, 49, 52]
    for string in first_dets_of_a_string:
        if string == 0:  continue
        if string == 52: continue
        plt.axvline(string-0.5, linestyle='--', color='gray')
    if peak == "K40_1461": y_text = 300
    if peak == "K42_1525": y_text = 410
    """
    plt.text(first_dets_of_a_string[1]-1, y_text, "1", color='gray', ha='center', va='center')
    plt.text(first_dets_of_a_string[2]-1, y_text, "2", color='gray', ha='center', va='center')
    plt.text(first_dets_of_a_string[3]-1, y_text, "3", color='gray', ha='center', va='center')
    plt.text(first_dets_of_a_string[4]-1, y_text, "4", color='gray', ha='center', va='center')
    plt.text(first_dets_of_a_string[5]-1, y_text, "5", color='gray', ha='center', va='center')
    plt.text(first_dets_of_a_string[6]-1, y_text, "7", color='gray', ha='center', va='center')
    plt.text(first_dets_of_a_string[7]-1, y_text, "8", color='gray', ha='center', va='center')
    plt.text(first_dets_of_a_string[8]-1, y_text, "9", color='gray', ha='center', va='center')
    plt.text(first_dets_of_a_string[9]-1, y_text, "10", color='gray', ha='center', va='center')
    plt.text(first_dets_of_a_string[10]-1, y_text, "11", color='gray', ha='center', va='center')
    """

    # Customize x-axis tick labels color based on the starting letter
    tick_labels = plt.xticks()[1]
    """
    for label in tick_labels:
        if label.get_text().startswith('V'):
            label.set_color(lps.colors["red"])
        elif label.get_text().startswith('B'):
            label.set_color(lps.colors["green"])
        elif label.get_text().startswith('C'):
            label.set_color(lps.colors["violet"])
        elif label.get_text().startswith('P'):
            label.set_color(lps.colors["orange"])
    """
    axes.set_legend_annotation()
    plt.xticks(rotation=90, ha='center')
    if peak == "K40_1461": plt.title(r'$^{40}$K - 1460.8 keV')
    if peak == "K42_1525": plt.title(r'$^{42}$K - 1524.7 keV')
    plt.ylabel("rate [counts/(kg yr)]")

    marker_proxy_dot = mlines.Line2D([], [], linestyle='-', marker='.', color='black', markersize=10, label='global mode +- 68%')
    marker_proxy_v = mlines.Line2D([], [], linestyle='None', marker='v', color='black', markersize=10, label='upper 90% CI limit')
    marker_proxy_dot_raw = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["raw"], markersize=10, label='raw')
    marker_proxy_v_raw = mlines.Line2D([], [], linestyle='None', marker='v', color=color_cut["raw"], markersize=10, label='raw')
    marker_proxy_dot_LArC = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["LArC"], markersize=10, label='LAr C')
    marker_proxy_v_LArC = mlines.Line2D([], [], linestyle='None', marker='v', color=color_cut["LArC"], markersize=10, label='LAr C')
    marker_proxy_dot_LArAC = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["LArAC"], markersize=10, label='LAr AC')
    marker_proxy_v_LArAC = mlines.Line2D([], [], linestyle='None', marker='v', color=color_cut["LArAC"], markersize=10, label='LAr AC')
    axes.legend([(marker_proxy_dot_raw, marker_proxy_v_raw), (marker_proxy_dot_LArAC, marker_proxy_v_LArAC), (marker_proxy_dot_LArC, marker_proxy_v_LArC), marker_proxy_dot, marker_proxy_v], 
            ["raw", 'LAr AC', "LAr C", 'global mode +- 68%', 'upper 90% limit'], 
            numpoints=1, 
            handler_map={tuple: HandlerTuple(ndivide=None)},
            loc = "upper left")

    if peak == "K40_1461": plt.ylim(-10,y_text+10)
    if peak == "K42_1525": plt.ylim(-15,y_text+15)
    #plt.show()
    if peak == "K40_1461":
        plt.savefig(f"potassium/{dataset}/K40_single.png")
        plt.savefig(f"potassium/{dataset}/K40_single.pdf")
    else:
        plt.savefig(f"potassium/{dataset}/K42_single.png")
        plt.savefig(f"potassium/{dataset}/K42_single.pdf")


# mean over string

In [ ]:
for peak in peaks:
    for my_cut in ["raw", "LArC", "LArAC"]:
        fix, axes = plt.subplots(figsize=(10,5))
        axes.set_legend_logo(position='upper right')
        df = single_df[(single_df.gamma == peak) & (single_df.cut == my_cut)].copy()

        df["yerr_low"] = df.apply(lambda row: max(row["intensity"] - row["range_min"], 0), axis=1)
        df["yerr_upp"] = df.apply(lambda row: max(-row["intensity"] + row["range_max"], 0), axis=1)
        df.loc[df["intensity"] == df["yerr_low"], "yerr_low"] = 0

        # order detectors based on their position/location in the array
        valid_detectors = [detector for detector in det_list if detector in df["detector"].values]
        # Convert the "detector" column to a categorical data type with the specified order
        df["detector"] = pd.Categorical(df["detector"], categories=valid_detectors, ordered=True)
        # Reorder the DataFrame based on the categorical order
        df = df.sort_values("detector")

        for string_det in df.string:
            df_string = df[df["string"]==string_det]
            # expo-weighted intensity
            intensity=0
            low_err=0
            upp_err=0
            for index, row in df_string.iterrows():
                intensity += row['intensity'] * row['expo']
                low_err += row['yerr_low'] * row['expo']
                upp_err += row['yerr_upp'] * row['expo']
            intensity /= df_string["expo"].sum()
            low_err /= df_string["expo"].sum()
            upp_err /= df_string["expo"].sum()
            
            if string_det == 1:
                plt.errorbar(
                    str(string_det),
                    intensity,
                    yerr=[[low_err], [upp_err]],
                    marker=".", linestyle="", color=color_cut[my_cut], markersize=5, label=legend_dict[my_cut], capsize=3
                )
            else:
                plt.errorbar(
                    str(string_det),
                    intensity,
                    yerr=[[low_err], [upp_err]],
                    marker=".", linestyle="", color=color_cut[my_cut], markersize=5, capsize=3
                )

        # delimit strings
        """
        first_dets_of_a_string = [0, 8, 10, 12, 19, 24, 30, 37, 40, 49, 52]
        for string in first_dets_of_a_string:
            if string == 0:  continue
            if string == 52: continue
            plt.axvline(string-0.5, linestyle='--', color='gray')
        """
        if peak == "K40_1461": 
            if my_cut == "raw": y_text = 280
            if my_cut == "LArC": y_text = 70
            if my_cut == "LArAC": y_text = 280
        if peak == "K42_1525":
            if my_cut == "raw": y_text = 410
            if my_cut == "LArC": y_text = 300
            if my_cut == "LArAC": y_text = 100

        # Customize x-axis tick labels color based on the starting letter
        tick_labels = plt.xticks()[1]
        """
        for label in tick_labels:
            if label.get_text().startswith('V'):
                label.set_color(lps.colors["red"])
            elif label.get_text().startswith('B'):
                label.set_color(lps.colors["green"])
            elif label.get_text().startswith('C'):
                label.set_color(lps.colors["violet"])
            elif label.get_text().startswith('P'):
                label.set_color(lps.colors["orange"])
        """
        axes.set_legend_annotation()
        #plt.xticks(rotation=90, ha='center')
        if peak == "K40_1461": plt.title(r'$^{40}$K - 1460.8 keV')
        if peak == "K42_1525": plt.title(r'$^{42}$K - 1524.7 keV')
        plt.ylabel("exposure weighted rate [counts/(kg yr)]")
        plt.xlabel("string")

        marker_proxy_dot = mlines.Line2D([], [], linestyle='-', marker='.', color='black', markersize=10, label='global mode +- 68%')
        marker_proxy_dot_raw = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["raw"], markersize=10, label='raw')
        marker_proxy_dot_LArC = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["LArC"], markersize=10, label='LAr C')
        marker_proxy_dot_LArAC = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["LArAC"], markersize=10, label='LAr AC')
        if my_cut == "raw": 
            axes.legend([(marker_proxy_dot_raw)], 
                    ["raw"], 
                    numpoints=1, 
                    handler_map={tuple: HandlerTuple(ndivide=None)},
                    loc = "upper left")
        if my_cut == "LArC":
            axes.legend([(marker_proxy_dot_LArC)], 
                    ["LAr C"], 
                    numpoints=1, 
                    handler_map={tuple: HandlerTuple(ndivide=None)},
                    loc = "upper left")
        if my_cut == "LArAC": 
            axes.legend([(marker_proxy_dot_LArAC)], 
                    ["LAr AC"], 
                    numpoints=1, 
                    handler_map={tuple: HandlerTuple(ndivide=None)},
                    loc = "upper left")

        if peak == "K40_1461": plt.ylim(-10,y_text+10)
        if peak == "K42_1525": plt.ylim(-15,y_text+15)
        #plt.show()
        if peak == "K40_1461":
            plt.savefig(f"potassium/{dataset}/K40_string_{my_cut}.png")
            plt.savefig(f"potassium/{dataset}/K40_string_{my_cut}.pdf")
        else:
            plt.savefig(f"potassium/{dataset}/K42_string_{my_cut}.png")
            plt.savefig(f"potassium/{dataset}/K42_string_{my_cut}.pdf")

In [ ]:
for peak in peaks:
    fix, axes = plt.subplots(figsize=(10,5))
    axes.set_legend_logo(position='upper right')
    for my_cut in ["raw", "LArC", "LArAC"]:
        df = single_df[(single_df.gamma == peak) & (single_df.cut == my_cut)].copy()

        df["yerr_low"] = df.apply(lambda row: max(row["intensity"] - row["range_min"], 0), axis=1)
        df["yerr_upp"] = df.apply(lambda row: max(-row["intensity"] + row["range_max"], 0), axis=1)
        df.loc[df["intensity"] == df["yerr_low"], "yerr_low"] = 0

        # order detectors based on their position/location in the array
        valid_detectors = [detector for detector in det_list if detector in df["detector"].values]
        # Convert the "detector" column to a categorical data type with the specified order
        df["detector"] = pd.Categorical(df["detector"], categories=valid_detectors, ordered=True)
        # Reorder the DataFrame based on the categorical order
        df = df.sort_values("detector")

        for string_det in df.string:
            df_string = df[df["string"]==string_det]
            # expo-weighted intensity
            intensity=0
            low_err=0
            upp_err=0
            for index, row in df_string.iterrows():
                intensity += row['intensity'] * row['expo']
                low_err += row['yerr_low'] * row['expo']
                upp_err += row['yerr_upp'] * row['expo']
            intensity /= df_string["expo"].sum()
            low_err /= df_string["expo"].sum()
            upp_err /= df_string["expo"].sum()
            
            if string_det == 1:
                plt.errorbar(
                    str(string_det),
                    intensity,
                    yerr=[[low_err], [upp_err]],
                    marker=".", linestyle="", color=color_cut[my_cut], markersize=5, label=legend_dict[my_cut], capsize=3
                )
            else:
                plt.errorbar(
                    str(string_det),
                    intensity,
                    yerr=[[low_err], [upp_err]],
                    marker=".", linestyle="", color=color_cut[my_cut], markersize=5, capsize=3
                )

        # delimit strings
        """
        first_dets_of_a_string = [0, 8, 10, 12, 19, 24, 30, 37, 40, 49, 52]
        for string in first_dets_of_a_string:
            if string == 0:  continue
            if string == 52: continue
            plt.axvline(string-0.5, linestyle='--', color='gray')
        """
        if peak == "K40_1461": 
            y_text = 280
        if peak == "K42_1525":
            y_text = 410
        # Customize x-axis tick labels color based on the starting letter
        tick_labels = plt.xticks()[1]
        """
        for label in tick_labels:
            if label.get_text().startswith('V'):
                label.set_color(lps.colors["red"])
            elif label.get_text().startswith('B'):
                label.set_color(lps.colors["green"])
            elif label.get_text().startswith('C'):
                label.set_color(lps.colors["violet"])
            elif label.get_text().startswith('P'):
                label.set_color(lps.colors["orange"])
        """
        axes.set_legend_annotation()
        #plt.xticks(rotation=90, ha='center')
        if peak == "K40_1461": plt.title(r'$^{40}$K - 1460.8 keV')
        if peak == "K42_1525": plt.title(r'$^{42}$K - 1524.7 keV')
        plt.ylabel("exposure weighted rate [counts/(kg yr)]")
        plt.xlabel("string")

        marker_proxy_dot = mlines.Line2D([], [], linestyle='-', marker='.', color='black', markersize=10, label='global mode +- 68%')
        marker_proxy_v = mlines.Line2D([], [], linestyle='None', marker='v', color='black', markersize=10, label='upper 90% CI limit')
        marker_proxy_dot_raw = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["raw"], markersize=10, label='raw')
        marker_proxy_v_raw = mlines.Line2D([], [], linestyle='None', marker='v', color=color_cut["raw"], markersize=10, label='raw')
        marker_proxy_dot_LArC = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["LArC"], markersize=10, label='LAr C')
        marker_proxy_v_LArC = mlines.Line2D([], [], linestyle='None', marker='v', color=color_cut["LArC"], markersize=10, label='LAr C')
        marker_proxy_dot_LArAC = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["LArAC"], markersize=10, label='LAr AC')
        marker_proxy_v_LArAC = mlines.Line2D([], [], linestyle='None', marker='v', color=color_cut["LArAC"], markersize=10, label='LAr AC')
        axes.legend([(marker_proxy_dot_raw), (marker_proxy_dot_LArAC), (marker_proxy_dot_LArC), marker_proxy_dot], 
            ["raw", 'LAr AC', "LAr C"], 
            numpoints=1, 
            handler_map={tuple: HandlerTuple(ndivide=None)},
            loc = "upper left")

        if peak == "K40_1461": plt.ylim(-10,y_text+10)
        if peak == "K42_1525": plt.ylim(-15,y_text+15)
        #plt.show()
        if peak == "K40_1461":
            plt.savefig(f"potassium/{dataset}/K40_string.png")
            plt.savefig(f"potassium/{dataset}/K40_string.pdf")
        else:
            plt.savefig(f"potassium/{dataset}/K42_string.png")
            plt.savefig(f"potassium/{dataset}/K42_string.pdf")


# mean over z-position
Evaluated starting from the full dataset (and not only the golden one!)

In [ ]:
with open('LGND_200_Baseline.json', 'r') as json_file:
    LGND_200_BSLN = json.load(json_file)

crys_assem = [k for k in LGND_200_BSLN.keys() if "Crys" in k]
crys_assem_dets = [d.split('CrysAssem')[-1] for d in crys_assem]
rod_heights = {crys_assem_dets[idx]: LGND_200_BSLN[c]['parameters']['rodheight'] for idx,c in enumerate(crys_assem)}
#rint(crys_assem)
#rint(crys_assem_dets)
print(rod_heights)

strings = [channel_map[k]["location"]["string"] for k,v in channel_map.items() if "location" in v and "string" in v["location"]]
strings = list(set(strings))
new_rod_heights = {}
for string in strings:
    position = [channel_map[k]["location"]["position"] for k,v in channel_map.items() if "location" in v and "string" in v["location"] and v["location"]["string"]==string]
    dets = [k for k,v in channel_map.items() if "location" in v and "string" in v["location"] and v["location"]["string"]==string]
    
    current_position = 0
    for det in dets:
        current_position += float(rod_heights[det])
        new_rod_heights.update({det: current_position})
print(new_rod_heights)

def get_z_pos(detector_key):
    return new_rod_heights[detector_key]

single_df['z_pos_mm'] = single_df['detector'].apply(get_z_pos)
single_df[(single_df.string==1) & (single_df.cut=="raw") & (single_df.gamma=="K40_1461")]

In [ ]:
all_rods = [v for v in new_rod_heights.values()] # full array
print("min:", min(all_rods))
print("max:", max(all_rods))
plt.hist(all_rods, bins=60)  
plt.axvline(x=130)
plt.axvline(x=300)
plt.axvline(x=450)
plt.axvline(x=730)
plt.xlabel('z position (mm)')
plt.ylabel('frequency')
plt.title('Full array - distribution of z positions')
#plt.show()
plt.savefig(f"potassium/z_pos.pdf")
plt.savefig(f"potassium/z_pos.png")

def determine_z_pos(z_pos_mm):
    if z_pos_mm <= 130:
        return 'top'
    if z_pos_mm > 130 and z_pos_mm <= 300:
        return 'mid-top'
    if z_pos_mm > 300 and z_pos_mm <= 450:
        return 'middle'
    if z_pos_mm > 450 and z_pos_mm <= 730:
        return 'mid-bottom'
    if z_pos_mm >= 730:
        return 'bottom'

single_df['z_pos'] = single_df['z_pos_mm'].apply(determine_z_pos)

In [ ]:
for k,z_pos_mm in new_rod_heights.items():
    if z_pos_mm >= 730:
        print(k, channel_map[k]["location"]["string"], channel_map[k]["location"]["position"])

In [ ]:
single_df[(single_df["gamma"] =="K40_1461") & (single_df["cut"] =="raw") & (single_df["z_pos"] =="mid-top") & (single_df["string"] ==1)]

In [ ]:
for peak in peaks:
    fix, axes = plt.subplots(figsize=(10,5))
    axes.set_legend_logo(position='upper right')
    for my_cut in ["raw", "LArC", "LArAC"]:
        df = single_df[(single_df.gamma == peak) & (single_df.cut == my_cut)].copy()

        df["yerr_low"] = df.apply(lambda row: max(row["intensity"] - row["range_min"], 0), axis=1)
        df["yerr_upp"] = df.apply(lambda row: max(-row["intensity"] + row["range_max"], 0), axis=1)
        df.loc[df["intensity"] == df["yerr_low"], "yerr_low"] = 0

        # order detectors based on their position/location in the array
        valid_detectors = [detector for detector in det_list if detector in df["detector"].values]
        # Convert the "detector" column to a categorical data type with the specified order
        df["detector"] = pd.Categorical(df["detector"], categories=valid_detectors, ordered=True)
        # Reorder the DataFrame based on the categorical order
        df = df.sort_values("detector")

        for string_det in df.z_pos:
            df_string = df[df["z_pos"]==string_det]
            # expo-weighted intensity
            intensity=0
            low_err=0
            upp_err=0
            for index, row in df_string.iterrows():
                intensity += row['intensity'] * row['expo']
                low_err += row['yerr_low'] * row['expo']
                upp_err += row['yerr_upp'] * row['expo']
            intensity /= df_string["expo"].sum()
            low_err /= df_string["expo"].sum()
            upp_err /= df_string["expo"].sum()
            
            if string_det == 1:
                plt.errorbar(
                    str(string_det),
                    intensity,
                    yerr=[[low_err], [upp_err]],
                    marker=".", linestyle="", color=color_cut[my_cut], markersize=5, label=legend_dict[my_cut], capsize=3
                )
            else:
                plt.errorbar(
                    str(string_det),
                    intensity,
                    yerr=[[low_err], [upp_err]],
                    marker=".", linestyle="", color=color_cut[my_cut], markersize=5, capsize=3
                )

    if peak == "K40_1461": 
        y_text = 170
    if peak == "K42_1525":
        y_text = 360
    # Customize x-axis tick labels color based on the starting letter
    tick_labels = plt.xticks()[1]
    axes.set_legend_annotation()
    #plt.xticks(rotation=90, ha='center')
    if peak == "K40_1461": plt.title(r'$^{40}$K - 1460.8 keV')
    if peak == "K42_1525": plt.title(r'$^{42}$K - 1524.7 keV')
    plt.ylabel("exposure weighted rate [counts/(kg yr)]")
    plt.xlabel("position in the array")

    marker_proxy_dot = mlines.Line2D([], [], linestyle='-', marker='.', color='black', markersize=10, label='global mode +- 68%')
    marker_proxy_v = mlines.Line2D([], [], linestyle='None', marker='v', color='black', markersize=10, label='upper 90% CI limit')
    marker_proxy_dot_raw = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["raw"], markersize=10, label='raw')
    marker_proxy_v_raw = mlines.Line2D([], [], linestyle='None', marker='v', color=color_cut["raw"], markersize=10, label='raw')
    marker_proxy_dot_LArC = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["LArC"], markersize=10, label='LAr C')
    marker_proxy_v_LArC = mlines.Line2D([], [], linestyle='None', marker='v', color=color_cut["LArC"], markersize=10, label='LAr C')
    marker_proxy_dot_LArAC = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["LArAC"], markersize=10, label='LAr AC')
    marker_proxy_v_LArAC = mlines.Line2D([], [], linestyle='None', marker='v', color=color_cut["LArAC"], markersize=10, label='LAr AC')
    axes.legend([(marker_proxy_dot_raw), (marker_proxy_dot_LArAC), (marker_proxy_dot_LArC), marker_proxy_dot], 
        ["raw", 'LAr AC', "LAr C"], 
        numpoints=1, 
        handler_map={tuple: HandlerTuple(ndivide=None)},
        loc = "upper left")

    if peak == "K40_1461": plt.ylim(-5,y_text+10)
    if peak == "K42_1525": plt.ylim(-5,y_text+15)
    #plt.show()
    if peak == "K40_1461":
        plt.savefig(f"potassium/{dataset}/K40_position.png")
        plt.savefig(f"potassium/{dataset}/K40_position.pdf")
    else:
        plt.savefig(f"potassium/{dataset}/K42_position.png")
        plt.savefig(f"potassium/{dataset}/K42_position.pdf")

In [ ]:
for peak in peaks:
    for my_cut in ["raw", "LArC", "LArAC"]:
        fix, axes = plt.subplots(figsize=(10,5))
        axes.set_legend_logo(position='upper right')
        df = single_df[(single_df.gamma == peak) & (single_df.cut == my_cut)].copy()

        df["yerr_low"] = df.apply(lambda row: max(row["intensity"] - row["range_min"], 0), axis=1)
        df["yerr_upp"] = df.apply(lambda row: max(-row["intensity"] + row["range_max"], 0), axis=1)
        df.loc[df["intensity"] == df["yerr_low"], "yerr_low"] = 0

        # order detectors based on their position/location in the array
        valid_detectors = [detector for detector in det_list if detector in df["detector"].values]
        # Convert the "detector" column to a categorical data type with the specified order
        df["detector"] = pd.Categorical(df["detector"], categories=valid_detectors, ordered=True)
        # Reorder the DataFrame based on the categorical order
        df = df.sort_values("detector")

        for string_det in df.z_pos:
            df_string = df[df["z_pos"]==string_det]
            # expo-weighted intensity
            intensity=0
            low_err=0
            upp_err=0
            for index, row in df_string.iterrows():
                intensity += row['intensity'] * row['expo']
                low_err += row['yerr_low'] * row['expo']
                upp_err += row['yerr_upp'] * row['expo']
            intensity /= df_string["expo"].sum()
            low_err /= df_string["expo"].sum()
            upp_err /= df_string["expo"].sum()
            
            if string_det == 1:
                plt.errorbar(
                    str(string_det),
                    intensity,
                    yerr=[[low_err], [upp_err]],
                    marker=".", linestyle="", color=color_cut[my_cut], markersize=5, label=legend_dict[my_cut], capsize=3
                )
            else:
                plt.errorbar(
                    str(string_det),
                    intensity,
                    yerr=[[low_err], [upp_err]],
                    marker=".", linestyle="", color=color_cut[my_cut], markersize=5, capsize=3
                )

        if peak == "K40_1461": 
            if my_cut == "raw": y_text = 150
            if my_cut == "LArC": y_text = 20
            if my_cut == "LArAC": y_text = 140
        if peak == "K42_1525":
            if my_cut == "raw": y_text = 380
            if my_cut == "LArC": y_text = 280
            if my_cut == "LArAC": y_text = 80

        # Customize x-axis tick labels color based on the starting letter
        tick_labels = plt.xticks()[1]
        axes.set_legend_annotation()
        #plt.xticks(rotation=90, ha='center')
        if peak == "K40_1461": plt.title(r'$^{40}$K - 1460.8 keV')
        if peak == "K42_1525": plt.title(r'$^{42}$K - 1524.7 keV')
        plt.ylabel("exposure weighted rate [counts/(kg yr)]")
        plt.xlabel("position in the array")

        marker_proxy_dot = mlines.Line2D([], [], linestyle='-', marker='.', color='black', markersize=10, label='global mode +- 68%')
        marker_proxy_dot_raw = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["raw"], markersize=10, label='raw')
        marker_proxy_dot_LArC = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["LArC"], markersize=10, label='LAr C')
        marker_proxy_dot_LArAC = mlines.Line2D([], [], linestyle='-', marker='.', color=color_cut["LArAC"], markersize=10, label='LAr AC')
        if my_cut == "raw": 
            axes.legend([(marker_proxy_dot_raw)], 
                    ["raw"], 
                    numpoints=1, 
                    handler_map={tuple: HandlerTuple(ndivide=None)},
                    loc = "upper left")
        if my_cut == "LArC":
            axes.legend([(marker_proxy_dot_LArC)], 
                    ["LAr C"], 
                    numpoints=1, 
                    handler_map={tuple: HandlerTuple(ndivide=None)},
                    loc = "upper left")
        if my_cut == "LArAC": 
            axes.legend([(marker_proxy_dot_LArAC)], 
                    ["LAr AC"], 
                    numpoints=1, 
                    handler_map={tuple: HandlerTuple(ndivide=None)},
                    loc = "upper left")

        if peak == "K40_1461": plt.ylim(0,y_text+10)
        if peak == "K42_1525": plt.ylim(0,y_text+15)
        #plt.show()
        if peak == "K40_1461":
            plt.savefig(f"potassium/{dataset}/K40_position_{my_cut}.png")
            plt.savefig(f"potassium/{dataset}/K40_position_{my_cut}.pdf")
        else:
            plt.savefig(f"potassium/{dataset}/K42_position_{my_cut}.png")
            plt.savefig(f"potassium/{dataset}/K42_position_{my_cut}.pdf")

In [ ]:
"""
single_df_long = pd.DataFrame()
single_df_long = single_df.copy()
# Function to extract the 'string' value from full_det_map
def get_location_string(detector):
    return full_det_map[detector]['location']['string']
def get_location_pos(detector):
    return full_det_map[detector]['location']['position']

single_df_long['string'] = single_df_long['detector'].apply(get_location_string)
single_df_long['position'] = single_df_long['detector'].apply(get_location_pos)


cbar_unit = "counts / (kg yr)"

for peak in ["K40_1461", "K42_1525"]:
    for my_cut in ["raw", "LArC", "LArAC"]:
        data_analysis = single_df_long[(single_df_long.gamma == peak) & (single_df_long.cut == my_cut)].copy()

        # drop duplicate rows, based on channel entry
        data_analysis = data_analysis.drop_duplicates(subset=["detector"])

        data_analysis = data_analysis.filter(
            ["detector", "intensity", "string", "position"]
        )

        # -------------------------------------------------------------------------------
        # OFF detectors
        # -------------------------------------------------------------------------------
        from legendmeta import JsonDB
        map_file = os.path.join("/lfs/l1/legend/data/public/prodenv/prod-blind/ref/v01.06", "inputs/dataprod/config")
        full_status_map = JsonDB(map_file).on(
            timestamp="20230701T000000Z", system="geds"
        )["analysis"]
        statuses = ["off", "ac", "no_psd"] if dataset=="golden" else ["off", "ac"]
        status_map = {key: value for key, value in full_status_map.items() if value.get('usability') in statuses}
        all_detectors = [det for det in status_map.keys() if "location" in status_map[det].keys() and "S" not in det]# remove SiPMs and AUX/DUMMY channels
        map_file = os.path.join("/lfs/l1/legend/data/public/prodenv/prod-blind/ref/v01.06", "inputs/hardware/configuration/channelmaps")
        channel_map = JsonDB(map_file).on(timestamp="20230701T000000Z")
        # keep only off,ac detectors
        filtered_channel_map = {key: value for key, value in channel_map.items() if key in status_map}
        filtered_channel_map = {k:v for k,v in filtered_channel_map.items() if "location" in v.keys() and "S" not in k} # remove SiPMs and AUX/DUMMY channels
        off_channels=list(filtered_channel_map.keys())

        if len(off_channels) != 0:
            for channel in off_channels:
                # check if the channel is already in the exposure dataframe; if not, add a new row for it
                if channel not in data_analysis["detector"].values:

                    # get position within the array + other necessary info
                    name, location, position = 0, filtered_channel_map[channel]["location"]["string"], filtered_channel_map[channel]["location"]["position"]

                    # define new row for not-ON detectors
                    new_row = [[channel, 0, location, position]]
                    new_df = pd.DataFrame(
                        new_row,
                        columns=["detector", "intensity", "string", "position"],
                    )
                    # add the new row to the dataframe
                    data_analysis = pd.concat(
                        [data_analysis, new_df], ignore_index=True, axis=0
                    )

        # values to plot
        result = data_analysis.pivot(
            index="position", columns="string", values="intensity"
        )
        result = result.round(3)

        # -------------------------------------------------------------------------------
        # plot
        # -------------------------------------------------------------------------------
        # create the figure
        fig = plt.figure(num=None, figsize=(8, 12), dpi=80, facecolor="w", edgecolor="k")
        sns.set(font_scale=1)

        # create labels for dets, with exposure values
        labels = result.astype(str)
        labels[result == 0] = ''

        # labels definition (AFTER having included OFF detectors too) -------------------------------
        # LOCATION:
        x_axis_labels = [f"S{no}" for no in sorted(data_analysis["string"].unique())]
        # POSITION:
        y_axis_labels = [
            no
            for no in range(
                min(data_analysis["position"].unique()),
                max(data_analysis["position"].unique() + 1),
            )
        ]
        
        if my_cut == "raw": cmap_col = "rocket"
        if my_cut == "LArC": cmap_col = "YlOrBr"
        if my_cut == "LArAC": cmap_col = "mako"

        # create the heatmap
        status_map = sns.heatmap(
            data=result,
            annot=labels,
            annot_kws={"size": 7},
            yticklabels=y_axis_labels,
            xticklabels=x_axis_labels,
            fmt="s",
            cbar=True,
            cbar_kws={"shrink": 0.3},
            linewidths=1,
            linecolor="white",
            square=True,
            rasterized=True,
            cmap=cmap_col,
        )

        # add title on top of the cbar
        plt.text(
            1.45,
            1,
            f"({cbar_unit})",
            transform=status_map.transAxes,
            horizontalalignment="center",
            verticalalignment="center",
        )

        plt.tick_params(
            axis="both",
            which="major",
            labelbottom=False,
            bottom=False,
            top=False,
            labeltop=True,
        )
        plt.yticks(rotation=0)
        title_font = {'weight': 'normal'}
        if peak == "K40_1461": status_map.set_title(r'$^{40}$K - 1460.8 keV', fontdict=title_font)
        if peak == "K42_1525": status_map.set_title(r'$^{42}$K - 1524.7 keV', fontdict=title_font)
        #plt.show()
        if peak == "K40_1461": 
            plt.savefig(f"potassium/{dataset}/K40_map_{my_cut}.pdf")
            plt.savefig(f"potassium/{dataset}/K40_map_{my_cut}.png")
        if peak == "K42_1525": 
            plt.savefig(f"potassium/{dataset}/K42_map_{my_cut}.pdf")
            plt.savefig(f"potassium/{dataset}/K42_map_{my_cut}.png")
"""

In [ ]:
lps.plot_colortable()